# ENSEMBLE MODEL


Make sure you add the models shared drive into MyDrive

In [1]:
%pip install torch torchvision torchaudio
%pip install pandas
%pip install scikit-learn
%pip install transformers


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import statistics
import torch
import re
import pickle
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os
import sklearn
import numpy as np

In [10]:
class Ensemble_Model():
  def __init__(self, text):
    self.text = text
    self.model = None
    self.mlp_classifier = None
    self.svm_classifier = None
    self.nb_classifier = None
    self.tfidf_vec = None
    self.tfidf_text = None
    self.cv_text = None
    self.cv = None
    self.tfidf = None
    self.tokenizer = None
    self.tfm_pred_label = None
    self.tfm_pred_score = None
    self.mlp_pred_label = None
    self.mlp_pred_score = None
    self.svm_pred_label = None
    self.svm_pred_score = None
    self.nb_pred_label = None
    self.nb_pred_score = None
    self.model_list = None

  def import_models(self):
    self.model = AutoModelForSequenceClassification.from_pretrained("large_files/pretrained models/transformer")
    self.tokenizer = AutoTokenizer.from_pretrained("large_files/pretrained models/transformer")

    with open('large_files/pretrained models/mlp.pkl', 'rb') as f:
      self.mlp_classifier = pickle.load(f)

    with open('large_files/pretrained models/tfidfMLP.pkl', 'rb') as f:
      self.tfidf_vec = pickle.load(f)

    with open('large_files/pretrained models/svm.pkl', 'rb') as f:
      self.svm_classifier = pickle.load(f)

    with open('large_files/pretrained models/vectorizer.pkl', 'rb') as f:
      self.cv = pickle.load(f)

    with open('large_files/pretrained models/tfidf.pkl', 'rb') as f:
      self.tfidf = pickle.load(f)

    with open('large_files/pretrained models/naivebayes.pkl', 'rb') as f:
      self.nb_classifier = pickle.load(f)

  def text_processing(self, text):
    input_text = re.sub(r'[\'"‘’“”]', '', text)
    input_text = re.sub('^.*\(Reuters\)\s*-\s*', '', input_text)
    input_text = [input_text]
    self.cv_text = self.cv.transform(input_text)
    self.tfidf_text = self.tfidf.transform(self.cv_text)
    return input_text

  def define_labels(prob):
    if prob >= 0.5:
      pred_class = "Real"  # Real news
    else:
      pred_class = "Fake"  # Fake news
    return pred_class

  def transformers(self,input_text):
    inputs = self.tokenizer(input_text, return_tensors="pt")

    # Forward pass through the model to obtain logits
    with torch.no_grad():
        outputs = self.model(**inputs)

    # Get the predicted class label
    tfm_pred = torch.sigmoid(outputs.logits).squeeze().tolist()  # Assuming binary classification
    if tfm_pred >= 0.5:
        self.tfm_pred_label = "Real"
    else:
        self.tfm_pred_label = "Fake"
    self.tfm_pred_score = torch.sigmoid(outputs.logits).item()
    return

  def mlp(self,input_text):
    input_tfidf = self.tfidf_vec.transform(input_text)
    mlp_prob = self.mlp_classifier.predict_proba(input_tfidf)
    self.mlp_pred_label = Ensemble_Model.define_labels(mlp_prob[0][1])
    self.mlp_pred_score = mlp_prob[0][1]
    return

  def svm(self,input_text):
    svm_prob = self.svm_classifier.decision_function(self.tfidf_text)
    self.svm_pred_label = Ensemble_Model.define_labels(1 / (1 + np.exp(-svm_prob))[0])
    self.svm_pred_score  = 1 / (1 + np.exp(-svm_prob))[0]
    return

  def naivebayes(self,input_text):
    nb_prob = self.nb_classifier.predict_proba(self.tfidf_text)
    self.nb_pred_label = Ensemble_Model.define_labels(nb_prob[0][1])
    self.nb_pred_score = nb_prob[0][1]
    return

  def predict_list(self):
    if self.nb_classifier == None:
      Ensemble_Model.import_models(self)
    text = Ensemble_Model.text_processing(self,self.text)
    Ensemble_Model.transformers(self,text)
    Ensemble_Model.mlp(self,text)
    Ensemble_Model.svm(self,text)
    Ensemble_Model.naivebayes(self,text)

    self.model_list = ["Transformers", "MLP", "SVM", "Naive Bayes"]
    self.label_list = [self.tfm_pred_label, self.mlp_pred_label, self.svm_pred_label, self.nb_pred_label]
    self.score_list = [self.tfm_pred_score, self.mlp_pred_score, self.svm_pred_score, self.nb_pred_score]
    return

  def predict_df(self):
    if self.model_list == None:
      Ensemble_Model.predict_list(self)

    result_df = pd.DataFrame({"Model": self.model_list,
                            "Label": self.label_list,
                            "Score": self.score_list})
    return result_df

  def predict(self):
    if self.model_list == None:
      Ensemble_Model.predict_list(self)

    fake_count = self.label_list.count("Fake")
    real_count = self.label_list.count("Real")
    if fake_count > real_count:
      result_label = "Fake"
      result_score = statistics.mean(self.score_list[i] for i in range(4) if self.score_list[i] < 0.5)
    elif fake_count < real_count:
      result_label = "Real"
      result_score = statistics.mean(self.score_list[i] for i in range(4) if self.score_list[i] > 0.5)
    else:
      result_score = statistics.mean(self.score_list[i] for i in range(4))
      if result_score >= 0.5:
        result_label = "Real"
      else:
        result_label = "Fake"

    string = f"Prediction: {result_label}\nAverage predicted score: {result_score}"
    return string

In [43]:
from tkinter import *
from tkinter import ttk
import sv_ttk

def predict_text(text):
    prediction_result = Ensemble_Model(f"""{text}""").predict()
    return prediction_result

def predict_df(text1):
    prediction_df_result = Ensemble_Model(f"""{text1}""").predict_df()
    return prediction_df_result

def on_focus_in(event):
    if text_input.get("1.0", "end-1c").strip() == "Enter your article text here...":
        text_input.delete("1.0", "end")
        text_input.config(fg='white')

def on_focus_out(event):
    if not text_input.get("1.0", "end-1c").strip():
        text_input.insert("1.0", "Enter your article text here...")
        text_input.config(fg='grey')

def on_predict():
    input_text = text_input.get("1.0", "end-1c")
    prediction_df_result = predict_df(input_text)
    
    for i in tree.get_children():
        tree.delete(i)
    
    for col in tree['columns']:
        tree.heading(col, text=col)
        tree.column(col, width=100, anchor='center')
    
    for row in prediction_df_result.itertuples(index=False, name=None):
        tree.insert('', 'end', values=row)
    
    result_label.config(text=predict_text(input_text))

root = Tk()
root.title("Model Prediction")
root.geometry("650x600")
sv_ttk.set_theme("dark")

style = ttk.Style()
style.configure("TButton", font=("Didot", 16))
style.configure("Treeview", font=("Didot", 16))
style.configure("TLabel", font=("Didot", 16)) 

frame = ttk.Frame(root)
frame.grid(row=0, column=0, sticky=(W, E, N, S), padx=20, pady=20)

frame.columnconfigure(0, weight=1)
frame.columnconfigure(1, weight=1)
frame.columnconfigure(2, weight=1)
frame.rowconfigure(0, weight=0)
frame.rowconfigure(1, weight=0)
frame.rowconfigure(2, weight=0)
frame.rowconfigure(3, weight=0)
frame.rowconfigure(4, weight=1)

title_label = ttk.Label(frame, text="Fake News Detector", font=("Didot", 20))
title_label.grid(row=0, column=0, columnspan=3, pady=(0,20), sticky="nsew")

tree = ttk.Treeview(frame, show="headings")
tree['columns'] = ("Model", "Label", "Score")
for col in tree['columns']:
    tree.heading(col, text=col)
    tree.column(col, anchor='center')
tree.grid(row=3, column=0, columnspan=3, pady=10, sticky='nsew')

text_input = Text(frame, height=10, width=50)
text_input.grid(row=0, column=0, columnspan=3, pady=10, padx=10, sticky="nsew")
text_input.config(highlightbackground="white", font="didot")
text_input.insert("1.0", "Enter your article text here...")
text_input.config(fg='grey')
text_input.bind("<FocusIn>", on_focus_in)
text_input.bind("<FocusOut>", on_focus_out)

predict_button = ttk.Button(frame, text="Predict", command=on_predict)
predict_button.grid(row=1, column=1, padx=10, pady=10, sticky="ew")

result_label = ttk.Label(frame, text="")
result_label.grid(row=2, column=0, columnspan=3, pady=(0,10), sticky="nsew")

root.mainloop()

/Users/aidanmurphy/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/aidanmurphy/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MLPClassifier from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/aidanmurphy/anaconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when us